In [1]:
import pandas as pd
import numpy as np
import time
import math
import os
import re
from pathlib import Path
import glob
from pandarallel import pandarallel
import sqlite3
import mysql.connector
from mysql.connector import Error
import unicodedata

import warnings
warnings.filterwarnings('ignore')

## 1. Processing WikiGazetteer

WikiGazetteer is a gazetteer based on Wikipedia and enriched with Geonames data. To build a WikiGazetteer into a MySQL database for a specific Wikipedia language and version, follow [these instructions](https://github.com/Living-with-machines/lwm_GIR19_resolving_places/tree/master/gazetteer_construction). This notebook assumes the user already has created WikiGazetteers for English, Spanish, and Greek, which are stored in a MySQL database.

### Create minimal Wikigazetteer

The following cells create a minimal WikiGazetteer. It results in a dataframe (which is stored in `toponym_matching/datasets/gazetteers/` as a pickle file), where each row corresponds to an altname-location pair. The dataframe has the following fields:
* **altname:** alternate name of a location.
* **pid:** persistent identifier of the location (e.g. the wikipedia title in WikiGazetteer).
* **lat:** latitude of the location.
* **lon:** longitude of the location.

In [2]:
def create_minimal_gaz(gazetteer):
    gazDB = ""
    cursorGaz = ""
    try:
        gazDB = mysql.connector.connect(
                host='localhost',
                database=gazetteer,
                user='testGazetteer',
                password='1234')
        if gazDB.is_connected():
            cursorGaz = gazDB.cursor(buffered=True)
    except Error as e:
        print("Error while connecting to MySQL", e)
        
    cursorGaz.execute("""SELECT altname, wiki_title, lat, lon FROM altname
                         JOIN location ON altname.main_id = location.id""")
    locs = cursorGaz.fetchall()
    df = pd.DataFrame(locs, columns =['altname', 'pid', 'lat', 'lon'])
    
    gaznames = {"wikiGazES":"wikigaz_es",
            "wikiGazEL":"wikigaz_el",
            "wikiGazetteer": "wikigaz_en"}
    df.to_pickle("../../datasets/gazetteers/" + gaznames[gazetteer] + ".pkl")

In [4]:
# Create minimal gazetteer: the argument is the name of the MySQL database
create_minimal_gaz("wikiGazES")
create_minimal_gaz("wikiGazEL")
create_minimal_gaz("wikiGazetteer")

### Find all distinct candidate mentions from gazetteer

Find all unique alternate names in the gazetteer (i.e. the candidate mentions), and format them according to the format needed as input for DeezyMatch:
```
Namps Maisnil	0	false
Municipio de Wedington	0	false
Corcuera	0	false
Sinématiali	0	false
Trakay	0	false
```

In [5]:
def format_for_candranker(gazname, unique_placenames_array):
    with open("../../datasets/candidate_mentions_sets/" + gazname + ".txt", "w") as fw:
        for pl in unique_placenames_array:
            pl = pl.strip()
            if pl:
                if not "wikipedia" in pl: # Remove altnames that are wikiURLs (from geonames)
                    if not any(char.isdigit() for char in pl):
                        if not '"' in pl:
                            fw.write(pl.strip() + "\t0\tfalse\n")

In [6]:
def find_placenames(gazetteer):
    gazDB = ""
    cursorGaz = ""
    try:
        gazDB = mysql.connector.connect(
                host='localhost',
                database=gazetteer,
                user='testGazetteer',
                password='1234')
        if gazDB.is_connected():
            cursorGaz = gazDB.cursor(buffered=True)
    except Error as e:
        print("Error while connecting to MySQL", e)
        
    cursorGaz.execute("""SELECT DISTINCT altname FROM altname""")
    unique_placenames = cursorGaz.fetchall()
    unique_placenames = [r[0] for r in unique_placenames]
    unique_placenames_array = list(set(list(np.array(unique_placenames))))
    return unique_placenames_array

In [7]:
# English WikiGazetteer:
wiki_en_array = find_placenames("wikiGazetteer")
format_for_candranker("wikigaz_en", wiki_en_array)

# Spanish WikiGazetteer:
wiki_es_array = find_placenames("wikiGazES")
format_for_candranker("wikigaz_es", wiki_es_array)

# Greek WikiGazetteer:
wiki_el_array = find_placenames("wikiGazEL")
format_for_candranker("wikigaz_el", wiki_el_array)

## 2. Processing Pleiades

Download Pleiades gazetteer [from here](http://atlantides.org/downloads/pleiades/dumps/pleiades-names-latest.csv.gz) and store it in `toponym_matching/resources/`.

In [8]:
df = pd.read_csv("../../resources/pleiades-names-latest.csv")

In [9]:
df.head()

,authors,bbox,created,creators,currentVersion,description,extent,id,locationPrecision,maxDate,...,pid,reprLat,reprLatLong,reprLong,tags,timePeriods,timePeriodsKeys,timePeriodsRange,title,uid
0,"Spann, P., R. Warner, R. Talbert, T. Elliott, ...","-3.606772, 39.460299, -3.606772, 39.460299",2010-09-24T19:02:22Z,P.O. Spann,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-3.606772, 3...",consabura,precise,640.0,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,NaN,HRL,"hellenistic-republican,roman,late-antique","-330.0,640.0",Consabura,fc0ee157ff11ce6d2e72cd7c5df06fee
1,"Spann, P., R. Warner, R. Talbert, T. Elliott, ...","-3.606772, 39.460299, -3.606772, 39.460299",2010-09-24T19:02:22Z,P.O. Spann,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-3.606772, 3...",consabrum,precise,640.0,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,NaN,HRL,"hellenistic-republican,roman,late-antique","-330.0,640.0",Consabrum,e2b8756302fb62ff0e301710c265a3e6
2,"Spann, P., R. Warner, R. Talbert, T. Elliott, ...","-3.606772, 39.460299, -3.606772, 39.460299",2010-09-24T19:02:22Z,P.O. Spann,NaN,NaN,"{""type"": ""Point"", ""coordinates"": [-3.606772, 3...",kondabora,precise,640.0,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,NaN,HRL,"hellenistic-republican,roman,late-antique","-330.0,640.0",Kondabora,f742b67b8343a6c68f222fbf607dcbf9
3,"Becker, J.","-4.0, 39.0, -3.0, 40.0",2016-02-08T23:21:01Z,jbecker,0.0,An ethnic name used by Pliny the Elder.,"{""type"": ""Polygon"", ""coordinates"": [[[-4.0, 39...",consaburrenses,precise,300.0,...,/places/265876,39.460299,"39.460299,-3.606772",-3.606772,NaN,R,roman,"-30.0,300.0",Consaburrenses,ddc6939eba8aaf3729e6c254dedc90e4
4,"Spann, P., R. Warner, R. Talbert, S. Gillies, ...","-2.0, 38.0, -1.0, 39.0",2010-09-24T19:02:24Z,P.O. Spann,1.0,NaN,"{""type"": ""Polygon"", ""coordinates"": [[[-2.0, 38...",contestania,rough,300.0,...,/places/265877,38.500000,"38.5,-1.5",-1.500000,NaN,HR,"hellenistic-republican,roman","-330.0,300.0",Contestania,f4d5e22c62f479e2f1241850023ab2c0


Filter only interested in entries written in Greek alphabet, and format them according to the format needed as input for DeezyMatch:

In [10]:
alternatename = []
pid = []
lat = []
lon = []

for i, row in df.iterrows():
    if row["nameLanguage"] == "grc" or row["nameLanguage"] == "el":
        if type(row["nameAttested"]) == str:
            toponym = row["nameAttested"]
            alternatename.append(toponym)
            pid.append(row["pid"])
            lat.append(row["reprLat"])
            lon.append(row["reprLong"])

In [11]:
pleiades_gaz = pd.DataFrame()
pleiades_gaz['altname'] = alternatename
pleiades_gaz['pid'] = pid
pleiades_gaz['lat'] = lat
pleiades_gaz['lon'] = lon

In [12]:
pleiades_gaz.to_pickle("../../datasets/gazetteers/pleiades.pkl")

unique_placenames_array = list(set(list(np.array(pleiades_gaz["altname"]))))
format_for_candranker("pleiades", unique_placenames_array)

### 2.1 Combine Pleiades and WikiGazEL

In [14]:
greek_wgaz_pleiades = pd.concat([pd.read_pickle("../../datasets/gazetteers/pleiades.pkl"), pd.read_pickle("../../datasets/gazetteers/wikigaz_el.pkl")])
greek_wgaz_pleiades = greek_wgaz_pleiades.drop_duplicates(subset = ['altname', 'lat', 'lon'])
greek_wgaz_pleiades.to_pickle("../../datasets/gazetteers/wikigaz_pleiades_el.pkl")

unique_placenames_array = list(set(list(np.array(greek_wgaz_pleiades["altname"].unique()))))
format_for_candranker("wikigaz_pleiades_el", unique_placenames_array)

## 3. Processing HGIS de las Indias

Do the following four steps only once:
1. Download gazetteer from https://dataverse.harvard.edu/file.xhtml?persistentId=doi:10.7910/DVN/FUSJD3/DK27GE&version=2.0
2. Unzip and file in `toponym_matching/resources/`.
3. Convert zip to df (uncomment and run cell below).
4. Store dataframe in `toponym_matching/resources/`.

In [15]:
## Uncomment and run this only once. Change filename accordingly:
# import simpledbf
# dbf = simpledbf.Dbf5('../../resources/gazetteer-2019-03-28/gazetteer-2019-03-28.dbf')
# df = dbf.to_dataframe()
# df.to_pickle("../../resources/hgis_de_las_indias.pkl")

In [ ]:
df = pd.read_pickle("../../resources/hgis_de_las_indias.pkl")
df.head()

In [ ]:
df_indias = pd.DataFrame()
df_indias["altname"] = df["label"]
df_indias["pid"] = df["gz_id"]
df_indias["lat"] = df["lat"]
df_indias["lon"] = df["lon"]

In [ ]:
df_indias = df_indias.drop_duplicates(subset=["altname", "pid", "lat", "lon"])

In [ ]:
df_indias.reset_index(drop=True, inplace=True)

In [ ]:
df_indias.head()

In [ ]:
df_indias.to_pickle("../../datasets/gazetteers/hgisindias.pkl")

unique_placenames_array = np.array(df_indias["altname"])
with open("../../datasets/candidate_mentions_sets/hgisindias.txt", "w") as fw:
    for pl in list(unique_placenames_array):
        pl = pl.strip()
        if pl:
            if not '"' in pl:
                fw.write(pl.strip() + "\t0\tfalse\n")

### 3.1. Combine HGISindias and WikiGazES

In [ ]:
wges = pd.read_pickle("../../datasets/gazetteers/wikigaz_es.pkl")
wges = wges.rename(columns={"wikititle": "pid"})

es_wgaz_hgisindias = pd.concat([pd.read_pickle("../../datasets/gazetteers/hgisindias.pkl"), wges])
es_wgaz_hgisindias = es_wgaz_hgisindias.drop_duplicates(subset = ['altname', 'lat', 'lon'])
es_wgaz_hgisindias.to_pickle("../../datasets/gazetteers/wikigaz_hgisindias_es.pkl")

unique_placenames_array = np.array(es_wgaz_hgisindias["altname"])
with open("../../datasets/candidate_mentions_sets/wikigaz_hgisindias_es.txt", "w") as fw:
    for pl in list(unique_placenames_array):
        pl = pl.strip()
        if pl:
            if not '"' in pl:
                fw.write(pl.strip() + "\t0\tfalse\n")